# Packages


In [2]:
import os, requests
import numpy as np
import matplotlib.pyplot as plt 
import pickle

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/') 


In [ ]:
# !ls "/content/drive/"
# %cd '/content/drive/My Drive/NMA_EasyCabbage'

# Data retrieval


In [4]:
# get file names
fname = []
for j in range(3):
    fname.append('steinmetz_part%d.npz'%j)
fname

['steinmetz_part0.npz', 'steinmetz_part1.npz', 'steinmetz_part2.npz']

In [8]:
# if nessescary: download data (now stored on the drive)
def download_data(fname):
    """
    This function downloads the data required datasets from the Steinmetz website

    Args:
        fname (list of str): filenames of required datasets

      """
    url = ["https://osf.io/agvxh/download"]
    url.append("https://osf.io/uv3mw/download")
    url.append("https://osf.io/ehmw2/download")
    for j in range(len(url)):
        if not os.path.isfile(fname[j]):
            try:
                r = requests.get(url[j])
            except requests.ConnectionError:
                print("!!! Failed to download data !!!")
            else:
                if r.status_code != requests.codes.ok:
                    print("!!! Failed to download data !!!")
                else:
                    with open(fname[j], "wb") as fid:
                        fid.write(r.content)

download_data(fname)

In [10]:
def load_data(fname):
    """
  This function loads data and stacks them horiontally

  Args:
      fname (list of str): file names of the required datasets

  Returns:
      (np.array): steinmetz data
      """
    alldat = np.array([])
    for j in range(len(fname)):
        alldat = np.hstack((alldat, np.load('data/'+ fname[j], allow_pickle=True)['dat']))

    return alldat

all_dat = load_data(fname)

# Constants

In [11]:
N_TIMEPOINTS = all_dat[0]['spks'].shape[-1]
BIN_SIZE_DATA = all_dat[0]['bin_size']
MAX_INTERVAL = N_TIMEPOINTS*BIN_SIZE_DATA

for neurons in all_dat:
    print(neurons['spks'].shape[1])

214
251
228
249
254
290
252
250
372
447
342
340
300
268
404
280
224
316
247
235
124
444
151
187
261
178
253
142
128
143
237
260
191
296
311
258
181
199
343


# Variable settings

In [13]:
# tested_bin_size
bin_size_ISI = 0.05

# Helper functions

In [21]:
def normalize(number_array):
    n = float(np.sum(number_array))
    if not n:
        return number_array, n
    return number_array/n, n

def get_ISI_trial(spike):
    spike_times = np.array([])
    for t,s in enumerate(spike):
        for i in range(s):
            spike_times = np.hstack([spike_times, [t]])

    spike_intervals = np.diff(spike_times) * BIN_SIZE_DATA
    spike_times = np.asarray(spike_times, dtype = float) * BIN_SIZE_DATA 
    return spike_intervals, spike_times

def get_ISI_neuron(neuron, trials):
    spike_intervals = np.array([])
  
    for trial in trials:
        spike = neuron[trial]
        spike_interval = get_ISI_trial(spike)
        spike_intervals = np.hstack([spike_intervals, spike_interval])
    return spike_intervals

def get_bins(neuron, bin_size_ISI, trials):
    spike_intervals = get_ISI_neuron(neuron, trials)
    spikes_per_bin, bins = np.histogram(spike_intervals, bins = np.arange(0, MAX_INTERVAL, bin_size_ISI))
    spikes_per_bin, n_spikes = normalize(spikes_per_bin)
    return spikes_per_bin, n_spikes


def get_n_dimensional_neurons(neurons, bin_size_ISI):
    neuron_amount = neurons.shape[0]
    n_bins = int(MAX_INTERVAL/bin_size_ISI) - 1
    n_dimensional_neurons = np.empty((neuron_amount, n_bins))
    rel_spikes = np.empty((neuron_amount,1))
    trials = [i for i in range(neurons.shape[1])]
    for i in range(neuron_amount):
        n_dimensional_neurons[i,:], rel_spikes[i,0] = get_bins(neurons[i], bin_size_ISI, trials)

    rel_spikes, total_spikes = normalize(rel_spikes)
    n_dimensional_neurons = np.hstack([n_dimensional_neurons, rel_spikes])

    return n_dimensional_neurons

In [ ]:
n_dim_bins_seperate_sessions = []

for i,session in enumerate(all_dat):
    neurons = session['spks']
    pca_input = get_n_dimensional_neurons(neurons, bin_size_ISI)
    print(pca_input)
    n_dim_bins_seperate_sessions.append(pca_input)

#     with open(f'bins_session{i}_b{bin_size_ISI}.pkl', 'wb') as f:
#         pickle.dump(pca_input, f)

    np.save(f'processed_data/bins_session{i}_b{bin_size_ISI}.npy', pca_input)

In [ ]:
bin_size_ISI = 0.02

n_dim_bins_seperate_sessions = []

for i,session in enumerate(all_dat):
    neurons = session['spks']
    pca_input = get_n_dimensional_neurons(neurons, bin_size_ISI)
    print(pca_input)
    n_dim_bins_seperate_sessions.append(pca_input)

#     with open(f'bins_session{i}_b{bin_size_ISI}.pkl', 'wb') as f:
#         pickle.dump(pca_input, f)

    np.save(f'processed_data/bins_session{i}_b{bin_size_ISI}.npy', pca_input)

In [ ]:
for i in range(len(all_dat)):
    np.save(f'processed_data/labels{i}.npy',all_dat[i]['brain_area'])

In [31]:
all_sessions = []

for i in range(len(all_dat)):
    neurons = all_dat[i]['spks']
    print(i)

    data = []
    data2 = []
    for neuron in neurons:

        #print(i)
        neuron_list = []
        neuron2 = []
        for trial in neuron:
            spike_ints, spike_times = get_ISI_trial(trial)
            neuron_list.append(spike_ints)
            neuron2.append(spike_times)
        neuron_list = np.asarray(neuron_list)
        data.append(neuron_list)
        neuron2 = np.asarray(neuron2)
        data2.append(neuron2)
        
    data = np.asarray(data)
    data2 = np.asarray(data2)
    
    all_session.append([data, data])
#     with open(f'bins_session{i}_b{bin_size_ISI}.pkl', 'wb') as f:
#         pickle.dump(pca_input, f)
    
    #np.save(f'session{i}_ISI.npy', data)
    #np.save(f'session{i}_spiketimes.npy', data)
    

0


KeyboardInterrupt: 

In [17]:
print(ISI)

NameError: name 'ISI' is not defined